## Clip Flood graph
remove disrupted edges that intersect simulated flooded areas

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np

In [3]:
from shapely.geometry import Point, shape
from shapely import geometry
import shapely

In [4]:
# Get reference to GOSTNets
import sys,os
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

In [5]:
# load graph
#G = nx.read_gpickle(os.path.join(r'temp', 'clipped_cap_haitien_walk_w_ferries_via_osmnx_w_time_adv_snap.pickle'))
G = nx.read_gpickle(os.path.join(r'temp', 'clipped_cap_haitien_walk_w_ferries_via_osmnx.pickle'))

### Fathom SSBN data was used as an input for the simulated flooded areas. The FU_1in50 layer and the P_1_in50 layers were merged, and in QGIS the raster calculater was used to filter raster pixels with a value greater than .25 meters, using a query similar to this:
### ("FD_1in20@1" >= 0.25) * 1"
### Then raster was polygonized using QGIS. Finally, the QGIS fix geometries tool was run.

In [6]:
import fiona
fc = fiona.open(r"C:\Users\war-machine\Documents\world_bank_work\haiti_gtfs_project\merged_P1in50_FD1_50.shp")

print(fc.schema)

#first feature of the shapefile
first = fc.next()
print(first) # (GeoJSON format)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

<ipython-input-6-a9e718b13178>:7: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  first = fc.next()


In [7]:
shp_geom = shape(first['geometry']) # or shp_geom = shape(first) with PyShp)
print(shp_geom)
print(type(shp_geom))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
#try using GOSTnets function instead
edges_gdf = gn.edge_gdf_from_graph(G)

In [9]:
edges_gdf[:3]

,stnode,endnode,osmid,ref,service,ferry,junction,maxspeed,oneway,name,access,bridge,lanes,length,highway,est_width,geometry
0,330725194,330743482,30034450,NaN,NaN,NaN,NaN,NaN,False,Route usine,NaN,NaN,NaN,527.985,unclassified,NaN,"LINESTRING (-72.17556 19.71362, -72.17558 19.7..."
1,330725194,2322274029,218402959,NaN,NaN,NaN,NaN,NaN,False,Route Boisdaut,NaN,NaN,NaN,245.988,unclassified,NaN,"LINESTRING (-72.17556 19.71362, -72.17535 19.7..."
2,330725194,2322274029,223315661,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,483.983,path,NaN,"LINESTRING (-72.17556 19.71362, -72.17565 19.7..."


### Find intersections between edges and flood polygons

In [10]:
intersection_list = []
#loop through each edge:
for index, row in edges_gdf.iterrows():
    #print(row['geometry'])
    line_geom = row['geometry']
    #loop through each polygon:
    for shapely_poly in fc:
        #print(shapely_poly)
        shapely_poly = shape(shapely_poly['geometry'])
        intersection_line = shapely_poly.intersection(line_geom)
        if intersection_line:
            #print("intersection found")
            #print(intersection_line)
            #print("osmid")
            #print(row['osmid'])
            #if true, add edge to a list of intersections
            intersection_list.append((row['stnode'],row['endnode']))

In [11]:
len(intersection_list)

872

In [12]:
# get rid of duplicates
intersection_list = list(set(intersection_list))

In [13]:
len(intersection_list)

866

In [14]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 4145
Number of edges: 11404
Average in degree:   2.7513
Average out degree:   2.7513


In [15]:
G_flooded = G.copy()

### need to remove disrupted edges from the graph
We need to remove the edges from the graph that intersect the flooded area. Then using only the walk/ferry graph we can run accessibility analysis and see how many routes are affected.

In [16]:
for line in intersection_list:
    G_flooded.remove_edge(line[0],line[1])

In [17]:
print(nx.info(G_flooded))

Name: 
Type: MultiDiGraph
Number of nodes: 4145
Number of edges: 10538
Average in degree:   2.5423
Average out degree:   2.5423


### need to take the largest connected graph and also lose some potentially dangling nodes 

In [18]:
# compatible with NetworkX 2.4
list_of_subgraphs = list(G_flooded.subgraph(c).copy() for c in nx.strongly_connected_components(G_flooded))
max_graph = None
max_edges = 0

# To sort the list in place...
list_of_subgraphs.sort(key=lambda x: x.number_of_edges(), reverse=True)

# for i in list_of_subgraphs:
#     print(f"The {i} graph contains {i.number_of_edges()} edges")
#     if i.number_of_edges() > max_edges:
#         max_edges = i.number_of_edges()
#         max_graph = i

# set your graph equal to the largest sub-graph
G_largest_flooded = list_of_subgraphs[0]

In [19]:
print(nx.info(G_largest_flooded))

Name: 
Type: MultiDiGraph
Number of nodes: 3733
Number of edges: 10216
Average in degree:   2.7367
Average out degree:   2.7367


In [20]:
G_largest_flooded.number_of_edges()

10216

In [21]:
len(list_of_subgraphs)

294

In [22]:
print(list_of_subgraphs[0].number_of_edges())

10216


In [23]:
print(list_of_subgraphs[1].number_of_edges())

266


In [24]:
print(list_of_subgraphs[2].number_of_edges())

16


In [25]:
print(list_of_subgraphs[3].number_of_edges())

16


In [26]:
print(list_of_subgraphs[4].number_of_edges())

6


### For visualization purposes save the top 3 largest disconnected graphs

In [31]:
#gn.save(G_largest_flooded,"flooded_clipped_cap_haitien_walk_w_ferries_via_osmnx_w_time_adv_snap", "temp",pickle = True, edges = True, nodes = False)
gn.save(G_largest_flooded,"flooded_clipped_cap_haitien_walk_w_ferries_via_osmnx", "temp",pickle = True, edges = True, nodes = True)

In [28]:
#gn.save(list_of_subgraphs[1],"cap_haitien_walk_w_ferries_flooded_largest_2", "temp",pickle = False, edges = True, nodes = False)

In [29]:
#gn.save(list_of_subgraphs[2],"cap_haitien_walk_w_ferries_flooded_largest_3", "temp",pickle = False, edges = True, nodes = False)

In [30]:
#gn.save(list_of_subgraphs[3],"cap_haitien_walk_w_ferries_flooded_largest_4", "temp",pickle = False, edges = True, nodes = False)